# LangGraph 单节点示例

In [ ]:
! pip install langgraph langsmith langchain-openai

In [ ]:
# langgraph chat
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph
from IPython.display import Image, display
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatOpenAI()

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chat(state: State):
    return {"messages":[llm.invoke(state["messages"])]}

# 单节点
workflow = StateGraph(State)
workflow.add_node(chat)
workflow.set_entry_point("chat")
workflow.set_finish_point("chat")
graph = workflow.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        break
    for event in graph.stream({"messages": ("user", user_input)}):
        for value in event.values():
            print("Assistant: ", value["messages"][-1].content)
